In [5]:
import matplotlib.pyplot as plt
import numpy as np
import math
import matplotlib.animation as animation
import matplotlib.patches as mpatches
import matplotlib.transforms as transforms
import random
from matplotlib.collections import PatchCollection
from matplotlib.colors import ListedColormap
from matplotlib import animation, rc
from IPython.display import HTML
maze1 = [[-20,-15],[-20,15],[-20,15],[20,15],[20,15],[20,-15],[20,-15],[-20,-15],
               [-10,-15],[-10,5],[-10,5],[-5,5],[-5,5],[-5,-15],
               [-10,15],[-10,10],[-10,10],[-5,10],[-5,10],[-5,15],
               [0,-15],[0,-10],[0,-10],[5,-10],[5,-10],[15,0],[15,0],[17,-5],[17,-5],[17,-13],
               [0,15],[0,-5],[0,-5],[5,-5],[5,-5],[5,15]]
def ccw(A,B,C):
    return (C[1]-A[1]) * (B[0]-A[0]) > (B[1]-A[1]) * (C[0]-A[0])
# Return true if line segments AB and CD intersect
def intersect_Q(A,B,C,D):
    return ccw(A,C,D) != ccw(B,C,D) and ccw(A,B,C) != ccw(A,B,D)
# Return minimum distance from C to line defined by A and B
def distance_min(A,B,C):
    return abs((B[0]-A[0])*(A[1]-C[1])-(A[0]-C[0])*(B[1]-A[1]))/math.sqrt((B[0]-A[0])**2+(B[1]-A[1])**2)
# Return distance from C to intersection of AB and CD
def distance(A,B,C,D):
    x1,y1,x2,y2,x3,y3,x4,y4 = A[0],A[1],B[0],B[1],C[0],C[1],D[0],D[1]
    Px = ((x1*y2-y1*x2)*(x3-x4)-(x1-x2)*(x3*y4-y3*x4))/((x1-x2)*(y3-y4)-(y1-y2)*(x3-x4))
    Py = ((x1*y2-y1*x2)*(y3-y4)-(y1-y2)*(x3*y4-y3*x4))/((x1-x2)*(y3-y4)-(y1-y2)*(x3-x4))
    return(math.sqrt((x3-Px)**2+(y3-Py)**2))
class robotic_car:
    def __init__(self,speed = 'fast'):
        self.path_x = [0]
        self.path_y = [0]
        self.headings = [0]
        self.speed = speed
        self.walls = [[-20,-15],[-20,15],[-20,15],[20,15],[20,15],[20,-15],[20,-15],[-20,-15]]
        self.crashed = False
        self.crashed_i = 0
        self.goal = False
        self.goalxy = []
        self.goal_reached = False
        global left,right,forward,back
        left = 1
        right = -1
        forward = 1
        back = -1
        
    def get_heading(self):
        return(np.rad2deg(self.headings[-1]))
    
    def get_position(self):
        return((self.path_x[-1],self.path_y[-1]))
    
    def turn(self,direction,angle):   
        if not self.crashed:
            i=0
            while i < angle: #turn 5 degree per frame ( turning over 5 degrees)
                if (angle-i)>=5:
                    delta_angle = direction*np.deg2rad(5)
                    i+=5
                else:
                    delta_angle=direction*np.deg2rad(1)
                    i+=1
                self.headings = np.append(self.headings,self.headings[-1]+delta_angle)
                self.headings = ( self.headings + np.pi) % (2 * np.pi ) - np.pi
                self.path_x = np.append(self.path_x,self.path_x[-1])
                self.path_y = np.append(self.path_y,self.path_y[-1])          

    def drive(self,direction,distance):  
        if not self.crashed:
            i=0
            start = self.get_position()
            start1 = [start[0]-1*np.sin(np.deg2rad(self.get_heading())),start[1]+1*np.cos(np.deg2rad(self.get_heading()))]        
            while i < distance and not self.crashed: #drive 0.3 units per frame
                scan = self.scan_ahead()
                heading = self.headings[-1]
                delta_x = -direction*0.3*np.sin(heading)
                delta_y = direction*0.3*np.cos(heading)
                self.headings = np.append(self.headings,self.headings[-1])
                self.path_x = np.append(self.path_x,self.path_x[-1]+delta_x)
                self.path_y = np.append(self.path_y,self.path_y[-1]+delta_y)
                stop = self.get_position()
                stop1 = [stop[0]-1*np.sin(np.deg2rad(self.get_heading())),stop[1]+1*np.cos(np.deg2rad(self.get_heading()))]        
                i+=0.3
                x = self.path_x[-1]
                y = self.path_y[-1]
                if self.goal and x>self.goalxy[0] and x<(self.goalxy[0]+5) and y>self.goalxy[1] and y<(self.goalxy[1]+5):
                    self.goal_reached=True
                #check if crashed
                if scan<100:
                    for j in range(0,len(self.walls),2):
                        wall_A = self.walls[j]
                        wall_B = self.walls[j+1]
                        if intersect_Q(wall_A,wall_B,start,stop) or intersect_Q(wall_A,wall_B,start1,stop1):
                            self.crashed = True
                            self.crashed_i = len(self.path_x)
                            for k in range(50):
                                self.path_x = np.append(self.path_x,self.path_x[-1])
                                self.path_y = np.append(self.path_y,self.path_y[-1])
                                self.headings = np.append(self.headings,self.headings[-1])
    def create_maze(self):
        self.path_x = [-15]
        self.path_y = [-10]
        self.headings = [0]
        self.walls = maze1
        self.goal = True
        self.goalxy = [0,-15]
              
    def scan_ahead(self):
        pos = self.get_position()
        heading = np.deg2rad(self.get_heading())
        self_A = pos
        self_B = [pos[0]-20*np.sin(heading),pos[1]+20*np.cos(heading)]
        ranges = [100]
        for i in range(0,len(self.walls),2):
            wall_A = self.walls[i]
            wall_B = self.walls[i+1]
            if intersect_Q(wall_A,wall_B,self_A,self_B):
                ranges=np.append(ranges,distance(wall_A,wall_B,self_A,self_B))
        return(min(ranges))
                
    def gen_walls(self,n_walls):
        while True:
            walls = [[-20,-15],[-20,15],[-20,15],[20,15],[20,15],[20,-15],[20,-15],[-20,-15]]
            for i in range(n_walls):
                start = np.array([random.randint(-20,20),random.randint(-15,15)])
                if random.randint(0,1)==1:
                    stop = start + np.array([random.randint(-10,10),0])
                else:
                    stop = start + np.array([0,random.randint(-10,10)])
                walls = np.concatenate((walls,[start],[stop]))
            self.walls = walls
            if self.scan_ahead()>3:
                break
    
    def set_goal(self,x,y):
        self.goal = True
        self.goalxy = [x-2.5,y-2.5]
    
    def get_goal_heading(self):
        position = self.get_position()
        if self.goal:
            goal = self.goalxy
            dx = goal[0]+2.5-position[0]
            dy = goal[1]+2.5-position[1]
            angle = -np.rad2deg(math.atan2(dx,dy))
            return(angle)
        
    def simulate(self):
        fig,ax = plt.subplots(figsize=(0.8*10,0.8*7.5))
        ax.set_xlim([-20,20])
        ax.set_ylim([-15,15])
        for i in range(0,len(self.walls),2):
            ax.plot([self.walls[i][0],self.walls[i+1][0]],[self.walls[i][1],self.walls[i+1][1]],zorder=0,color='g',lw=5)
        line,=ax.plot(0,0,zorder=0)
        if self.goal:
            goal = mpatches.Rectangle(self.goalxy,5,5,zorder=0,color='g',alpha=0.5)
            ax.add_patch(goal)
        L_wheel = mpatches.Rectangle([-0.9,-0.2],0.3,1,color='k')
        R_wheel = mpatches.Rectangle([0.6,-0.2],0.3,1,color='k')
        eye1 = mpatches.Circle([-0.25,1],0.2,ec = 'k',fc='w')
        eye2 = mpatches.Circle([0.25,1],0.2,ec = 'k',fc='w')
        eyep1 = mpatches.Circle([-0.28,1.1],0.05,color='k')
        eyep2 = mpatches.Circle([0.28,1.1],0.05,color='k')
        body = mpatches.Rectangle([-0.5,-1],1,2,fc='orange')     
        patches = [L_wheel,R_wheel,body,eye1,eye2,eyep1,eyep2]
        p = PatchCollection(patches,match_original=True)
        ax.add_collection(p)
        def animate(i):      
            line.set_data(self.path_x[1:i],self.path_y[1:i])
            x = self.path_x[i]
            y = self.path_y[i]
            heading = self.headings[i]
            p.set_transform(transforms.Affine2D().rotate(heading)+transforms.Affine2D().translate(x,y)+ ax.transData)
            if self.crashed and i>self.crashed_i:
                ax.annotate("CRASHED",(-19.5,-5),fontsize=90,c='r',style='italic')
            if self.goal and x>self.goalxy[0] and x<(self.goalxy[0]+5) and y>self.goalxy[1] and y<(self.goalxy[1]+5):
                self.goal = False
                self.goalxy=[]
                goal.remove()
            return p,line
        plt.close()
        if self.speed == 'fast':
            frame_time = 50
        elif self.speed == 'superfast':
            frame_time = 20
        else:
            frame_time = 100
        ani = animation.FuncAnimation(fig, animate, len(self.path_x),interval=frame_time, blit=True,repeat=False)
        return(HTML(ani.to_jshtml()))

SyntaxError: cannot assign to operator (3834248094.py, line 147)